## Prediction with a trained model of a single sentence

## Preparation

In [14]:
import numpy as np
import pandas as pd
import torch
import mlflow.pytorch

from basic_functions import (
    tokenize,
)

In [15]:
def get_tokenized_text(txt):
    x_tokenized = tokenize(txt, "distilbert-base-uncased")
    return x_tokenized

def predict(model, encodings, batch_size=8):
    # Set the model to evaluation mode
    model.eval()
    
    # Use GPU
    device = torch.device("mps")
    model.to(device)
    
    # Perform inference
    probabilities = []
    for i in range(0, len(encodings["input_ids"]), batch_size):
        with torch.no_grad():
            batch = {
                key: val[i:i+batch_size].to(device) 
                for key, val in encodings.items()
            }
            outputs = model(**batch)
            probs = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
            probabilities.extend(probs)
            
        # Clear GPU memory after each batch
        torch.mps.empty_cache()
    
    return np.array(probabilities)

def get_first_prediction(proba):
    pred = np.argmax(proba, axis=1)
    pred_int = int(pred[0])
    proba = proba[np.arange(len(pred)), pred]
    proba_int = float(proba[0])
    return pred_int, proba_int

def get_second_prediction(proba):
    pred = np.argsort(proba, axis=1)[:, -2] 
    pred_int = int(pred[0])
    proba = np.sort(proba, axis=1)[:, -2]
    proba_int = float(proba[0])
    return pred_int, proba_int

In [16]:
model = mlflow.pytorch.load_model('../models/distilbert_multiclass_with_none/pytorch_model')
    

## Testing

In [34]:
###Insert your Text here:
txt = 'yes'


### prediction
tokenized_txt = get_tokenized_text(txt)
probabilities = predict(model, tokenized_txt)
fallacies = ['ad_hominem', 'appeal_to_authority', 'appeal_to_emotion', 'false_dilemma', 'slippery_slope', 'none']
df = pd.DataFrame({'fallacy': fallacies, 'probability': probabilities[0]})
print('All probabilites:', probabilities)
df.sort_values(by='probability', ascending=False)

All probabilites: [[0.03451002 0.00766116 0.9418941  0.00486484 0.00827857 0.00279129]]


,fallacy,probability
2,appeal_to_emotion,0.941894
0,ad_hominem,0.034510
4,slippery_slope,0.008279
1,appeal_to_authority,0.007661
3,false_dilemma,0.004865
5,none,0.002791
